In [1]:
import ee

ee.Authenticate()
ee.Initialize()

Get the most cloudy image in Jakarta

In [2]:
sentinel_2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
geometry = ee.Geometry.BBox(106.7537, -6.2624, 106.8992, -6.12049) # Jakarta

sentinel_2 = (
    sentinel_2
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 35))
    .filter(ee.Filter.date('2019-01-01', '2020-01-01'))
    .filter(ee.Filter.bounds(geometry))
)
sentinel_2 = sentinel_2.sort('CLOUDY_PIXEL_PERCENTAGE', False)
sample_s2_image = sentinel_2.first().clip(geometry)

In [3]:
import geemap

rgbVis = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}
Map = geemap.Map()
Map.addLayer(sample_s2_image, rgbVis, 'Sentinel-2')
Map.centerObject(geometry)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Cloud masking

In [4]:
def mask_s2_cloud(image: ee.Image) -> ee.Image:
    qa60 = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa60.bitwiseAnd(cloudBitMask).eq(0).And(qa60.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).select('B.*').copyProperties(image, ['system:time_start'])

s2_masked = sentinel_2.map(mask_s2_cloud)
Map = geemap.Map()
Map.addLayer(s2_masked.first().clip(geometry), rgbVis, 'Sentinel-2 Masked')
Map.centerObject(geometry)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…